<a href="https://colab.research.google.com/github/Noy444/Automated-Data-Analysis/blob/main/Automated_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Automated Data Analysis**
The code is divided into two main parts for your convenience:
1. **Data Input:** In the first part, you will provide the necessary data inputs.
2. **Visualization:** After providing the data, **you can run the second part multiple times to generate various visualizations of your data**.

Make sure that:

*   Your data is clean and free from inconsistencies.
*   All variables are categorized consistently and accurately (e.g., uniform spellings and capitalization).
*   The Excel file is correctly formatted with the appropriate sheet and column names.

This will ensure the automation runs smoothly and provides accurate results.



In [ ]:
# @title Step 1: Insert data & define variables
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from IPython.display import display, Markdown
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from google.colab import files


# Import Excel data and choose the relevant columns
print("Ensure that the relevant columns in your data file contain only necessary information.")
uploaded = files.upload()
relevant_sheet_name = input("Enter the sheet name: ")

data_file = pd.read_excel(next(iter(uploaded)), sheet_name=relevant_sheet_name)

print("Now You are about to enter two relevant columns.")
relevant_first_column = input("Enter the independent variable column name: ")
relevant_second_column = input("Enter the dependent variable column name: ")

# Ask the user if any of the columns are numeric
is_first_column_numeric = input(f"Is the independent variable '{relevant_first_column}' numeric? (yes/no): ").strip().lower()
is_second_column_numeric = input(f"Is the dependent variable '{relevant_second_column}' numeric? (yes/no): ").strip().lower()

if is_first_column_numeric == "yes":
    data_file[relevant_first_column] = data_file[relevant_first_column].astype('category')

if is_second_column_numeric == "yes":
    data_file[relevant_second_column] = data_file[relevant_second_column].astype('category')

# Reverses Hebrew characters if exist; leaves English values unchanged
def is_hebrew(s):
    return any("\u05D0" <= c <= "\u05EA" for c in str(s))

def reverse_hebrew_values(column):
    return [s[::-1] if is_hebrew(s) else s for s in column]

if is_first_column_numeric == "no":
    data_file[relevant_first_column] = reverse_hebrew_values(data_file[relevant_first_column])

if is_second_column_numeric == "no":
    data_file[relevant_second_column] = reverse_hebrew_values(data_file[relevant_second_column])

data_file = data_file.dropna(subset=[relevant_first_column, relevant_second_column])

# Calculate count, total, and percentage for charts
count_data = data_file.groupby([relevant_first_column, relevant_second_column], observed=True).size().reset_index(name='Count')
total_counts = count_data.groupby(relevant_first_column)['Count'].sum().reset_index(name='Total')
count_data = pd.merge(count_data, total_counts, on=relevant_first_column)
count_data['Percentage'] = (count_data['Count'] / count_data['Total']) * 100

display(Markdown("<b>Thank you! All data has been successfully received.</b><br><b>You can now proceed to the next step to choose a plot type.</b>"))

In [ ]:
# @title Step 2: Choose plot type
# Prompt user for plot type and plotting preferences
plot_type = input("Which plot would you like to generate? (bar/plot/pie): ").strip().lower()

if plot_type in ['bar', 'plot']:
    plot_all_in_one = input("Display all categories in one plot or separate plots? (one/separate): ").strip().lower()
else:
    plot_all_in_one = 'separate'

# Loop through each unique category in the second column
categories = count_data[relevant_second_column].unique()
colors = plt.colormaps['Set2']
bar_width = 0.8 / len(categories)
index = range(len(count_data[relevant_first_column].unique()))

# Plotting logic will be adjusted to work directly with count_data
if plot_all_in_one == 'one':
    fig, ax = plt.subplots(figsize=(12, 8))

    if plot_type == 'bar':
        for i, category in enumerate(categories):
            subset = count_data[count_data[relevant_second_column] == category]
            color = colors(i % len(colors.colors))
            ax.bar([x + i * bar_width for x in index], subset['Percentage'], bar_width, label=category, color=color)
        ax.set_xlabel(relevant_first_column)
        ax.set_ylabel('Percentage')
        ax.set_title(f'Average Percentage of {relevant_second_column} in Each {relevant_first_column}', fontweight='bold')
        ax.legend(loc='upper left', bbox_to_anchor=(1, 1))

        if is_first_column_numeric == "no":
            ax.set_xticks(range(len(subset[relevant_first_column])))
            ax.set_xticklabels(subset[relevant_first_column], rotation=45, ha='right')
        plt.show()

    elif plot_type == 'plot':
        for i, category in enumerate(categories):
            subset = count_data[count_data[relevant_second_column] == category]
            color = colors(i % len(colors.colors))
            ax.plot(subset[relevant_first_column], subset['Percentage'], label=category, color=color)
        ax.set_xlabel(relevant_first_column)
        ax.set_ylabel('Percentage')
        ax.set_title(f'Line Plot of {relevant_first_column} by {relevant_second_column}', fontweight='bold')
        ax.legend(loc='upper left', bbox_to_anchor=(1, 1))

        if is_first_column_numeric == "no":
            ax.set_xticks(range(len(subset[relevant_first_column])))
            ax.set_xticklabels(subset[relevant_first_column], rotation=45, ha='right')
            plt.show()

    else:
        print("Invalid plot type selected.")

elif plot_all_in_one == 'separate':
    for i, category in enumerate(categories):
        subset = count_data[count_data[relevant_second_column] == category]
        color = colors(i % len(colors.colors))
        if plot_type == 'bar':
            fig, ax = plt.subplots(figsize=(12, 8))
            ax.bar(subset[relevant_first_column], subset['Percentage'], color=color)
            ax.set_xlabel(relevant_first_column)
            ax.set_ylabel('Percentage')
            ax.set_title(f'Bar Plot of {category} in Each {relevant_first_column}', fontweight='bold')

            if is_first_column_numeric == "no":
                ax.set_xticks(range(len(subset[relevant_first_column])))
                ax.set_xticklabels(subset[relevant_first_column], rotation=45, ha='right')
            plt.show()

        elif plot_type == 'plot':
            fig, ax = plt.subplots(figsize=(12, 8))
            ax.plot(subset[relevant_first_column], subset['Percentage'], color=color)
            ax.set_xlabel(relevant_first_column)
            ax.set_ylabel('Percentage')
            ax.set_title(f'Line Plot of {category} by {relevant_first_column}', fontweight='bold')

            if is_first_column_numeric == "no":
                ax.set_xticks(range(len(subset[relevant_first_column])))
                ax.set_xticklabels(subset[relevant_first_column], rotation=45, ha='right')
            plt.show()

        elif plot_type == 'pie':
            if is_first_column_numeric == 'yes':
                print("Pie charts can't be created for numeric independent variables.")
                break

            fig, ax = plt.subplots(figsize=(12, 6))
            non_zero_subset = subset[subset['Count'] > 0]
            missing_categories = subset[subset['Count'] == 0][relevant_first_column].tolist()

            def reverse_hebrew_in_list(lst):
                return [s[::-1] if is_hebrew(s) else s for s in lst]

            missing_categories_str = ", ".join(reverse_hebrew_in_list(missing_categories))

            if len(non_zero_subset) > 0:
                ax.pie(non_zero_subset['Percentage'], labels=non_zero_subset[relevant_first_column], autopct='%1.1f%%', startangle=140)
                ax.set_title(f'Pie Chart of {category}', fontweight='bold')
            else:
                ax.set_title(f"No data available for {category}", fontweight='bold')

            plt.tight_layout()
            plt.show()

            # Show missing categories underneath the pie chart
            display(Markdown(f"**Categories absent in the dependent variable:** {missing_categories_str if missing_categories else 'None'}"))

        else:
            print("Invalid plot type selected.")

else:
    print("Invalid choice for plot display.")

# Create new column names without spaces
no_spaces_first_column_name = relevant_first_column.replace(' ', '')
no_spaces_second_column_name = relevant_second_column.replace(' ', '')

# Rename the columns inside the data_file
data_file = data_file.rename(columns={
    relevant_first_column: no_spaces_first_column_name,
    relevant_second_column: no_spaces_second_column_name
})

# Process data based on the column types for ANOVA and Post Hoc
count_df = data_file.groupby([no_spaces_first_column_name, no_spaces_second_column_name]).size().reset_index(name='Count')
model = ols(f"Count ~ C({no_spaces_second_column_name}) + C({no_spaces_first_column_name})", data=count_df).fit()
anova_table_raw = sm.stats.anova_lm(model, type=2)
anova_table_styled = anova_table_raw.style.map(lambda p: 'background-color: #80aaff' if p < 0.05 else '', subset=['PR(>F)'])

display(Markdown("## **ANOVA Results:**"))
display(anova_table_styled)
print('Significant cells are highlighted in color.\n')

relevant_second_column_p_value = anova_table_raw.at[f'C({no_spaces_second_column_name})', 'PR(>F)']
relevant_first_column_p_value = anova_table_raw.at[f'C({no_spaces_first_column_name})', 'PR(>F)']

# Perform Post Hoc tests
if relevant_second_column_p_value < 0.05:
    display(Markdown(f"## **Post hoc test for significant {relevant_second_column} variable:**"))
    tukey_results_prof = pairwise_tukeyhsd(endog=count_df['Count'], groups=count_df[no_spaces_second_column_name], alpha=0.05)
    tukey_results_prof_df = pd.DataFrame(tukey_results_prof.summary().data[1:], columns=tukey_results_prof.summary().data[0])

    # Filter results where reject is True and False
    reject_true_df = tukey_results_prof_df[tukey_results_prof_df['reject'] == True]
    reject_false_df = tukey_results_prof_df[tukey_results_prof_df['reject'] == False]

    # Function to highlight reject column
    def highlight_reject(val):
        return 'background-color: #80aaff' if val == True else ''

    if not reject_true_df.empty:
        reject_true_styled = reject_true_df.style.map(highlight_reject, subset=['reject'])
        display(Markdown("**Reject = True:**"))
        display(reject_true_styled)
    else:
        display("No relevant cases for this scenario (Reject = True)")

    if not reject_false_df.empty:
        reject_false_styled = reject_false_df.style.map(highlight_reject, subset=['reject'])
        display(Markdown("**Reject = False:**"))
        display(reject_false_styled)
    else:
        display("No relevant cases for this scenario (Reject = False)")

if relevant_first_column_p_value < 0.05:
    display(Markdown(f"## **Post hoc test for significant {relevant_first_column} variable:**"))
    tukey_results_first = pairwise_tukeyhsd(endog=count_df['Count'], groups=count_df[no_spaces_first_column_name], alpha=0.05)
    tukey_results_first_df = pd.DataFrame(tukey_results_first.summary().data[1:], columns=tukey_results_first.summary().data[0])

    # Filter results where reject is True and False
    reject_true_df = tukey_results_first_df[tukey_results_first_df['reject'] == True]
    reject_false_df = tukey_results_first_df[tukey_results_first_df['reject'] == False]

    # Function to highlight reject column
    def highlight_reject(val):
        return 'background-color: #80aaff' if val == True else ''

    if not reject_true_df.empty:
        reject_true_styled = reject_true_df.style.map(highlight_reject, subset=['reject'])
        display(Markdown("**Reject = True:**"))
        display(reject_true_styled)
    else:
        display(f"No relevant cases for this scenario (Reject = True)")

    if not reject_false_df.empty:
        reject_false_styled = reject_false_df.style.map(highlight_reject, subset=['reject'])
        display(Markdown("**Reject = False:**"))
        display(reject_false_styled)
    else:
        display(f"No relevant cases for this scenario (Reject = False)")